In [ ]:
#Question #1: 

In [12]:
import requests
from bs4 import BeautifulSoup

def search_amazon_product(product):
    query = product.replace(' ', '+')
    url = f'https://www.amazon.in/s?k={query}'

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')

    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    for product in products:
        title = product.find('h2').text.strip()

        price = product.find('span', {'class': 'a-price-whole'})
        if price:
            price = price.text.strip()
        else:
            price = 'Not available'

        rating = product.find('span', {'class': 'a-icon-alt'})
        if rating:
            rating = rating.text.strip()
        else:
            rating = 'No rating'

        print('Title:', title)
        print('Price:', price)
        print('Rating:', rating)
        print()

search_term = input("Enter the product to search: ")

search_amazon_product(search_term)



Enter the product to search: guitar
Title: Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black
Price: 1,994
Rating: 4.0 out of 5 stars

Title: Yamaha F280 Acoustic Rosewood Guitar (Natural, Beige)
Price: Not available
Rating: 4.3 out of 5 stars

Title: Juarez Lindenwood Acoustic Guitar Kit, 38 Inches Cutaway, 38C With Bag, Strings, Pick And Strap, 3TS Sunburst
Price: 1,994
Rating: 4.0 out of 5 stars

Title: Juarez Lindenwood Acoustic Guitar Kit, 38 Inches Cutaway, 38C With Bag, Strings, Pick And Strap, TBS Transparent (Blue Sunburst)
Price: 1,994
Rating: 4.0 out of 5 stars

Title: Intern 38C Acoustic Steel-string Guitar Premium White Cutaway Design with carry bag, strings, guitar strap and plectrums (INT-38C-WH)
Price: 2,249
Rating: 4.0 out of 5 stars

Title: Henrix 38C 38 Inch Cutaway Acoustic Guitar With Dual Action Truss Rod, Gigbag, Picks, String Set, String Winder, Strap, Polishing Cloth & Ebook - Black
Price: 2,999
Rating: 3.8 out of 5 stars

Ti

In [ ]:
#Question #2:

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_details(product):
    query = product.replace(' ', '+')
    url = f'https://www.amazon.in/s?k={query}'

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')

    total_pages = get_total_pages(soup)

    products = []
    for page in range(1, total_pages + 1):
        page_url = f'https://www.amazon.in/s?k={query}&page={page}'
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.text, 'html.parser')
        products += scrape_products_from_page(page_soup)

    df = pd.DataFrame(products, columns=[
        'Brand Name',
        'Name of the Product',
        'Price',
        'Return/Exchange',
        'Expected Delivery',
        'Availability',
        'Product URL'
    ])

    df.to_csv('product_details.csv', index=False)

    print("Product details have been scraped and saved to 'product_details.csv'.")

def get_total_pages(soup):
    pagination = soup.find('div', {'class': 's-pagination'})
    if pagination is None:
        return 1
    page_spans = pagination.find_all('span', {'class': 's-pagination-item'})
    return int(page_spans[-2].text)

def scrape_products_from_page(soup):
    products = []

    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for result in results:
        title_element = result.find('h2')
        title = title_element.text.strip() if title_element else '-'

        price_element = result.find('span', {'class': 'a-price-whole'})
        price = price_element.text.strip() if price_element else '-'

        return_exchange_element = result.find('div', {'class': 's-is-o'})
        return_exchange = return_exchange_element.text.strip() if return_exchange_element else '-'

        delivery_element = result.find('span', {'class': 's-icon'})
        delivery = delivery_element.text.strip() if delivery_element else '-'

        availability_element = result.find('span', {'class': 'a-size-medium'})
        availability = availability_element.text.strip() if availability_element else '-'

        product_url = result.find('a', {'class': 'a-link-normal'})
        product_link = f"https://www.amazon.in{product_url['href']}" if product_url else '-'

        brand_name = result.find('span', {'class': 'a-size-base-plus'})
        brand = brand_name.text.strip() if brand_name else '-'

        products.append([
            brand,
            title,
            price,
            return_exchange,
            delivery,
            availability,
            product_link
        ])

    return products

search_term = input("Enter the product to search: ")

scrape_product_details(search_term)

Enter the product to search: guitar
Product details have been scraped and saved to 'product_details.csv'.


In [20]:
df=pd.read_csv('product_details.csv')
df


,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL


In [22]:
#question #3

In [34]:
pip install googlesearch-python

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4211 sha256=5d359ecb61863f169840278c5a34bbae3c16a403174d004123835e40182d15c7
  Stored in directory: c:\users\q hammad\appdata\local\pip\cache\wheels\be\78\a3\d80e85ac9551489cd9c44f59a0bc5972e79e680ce9cc6055ca
Successfully built googlesearch-python
Note: you may need to restart the kernel to use updated packages.


In [46]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import os

def scrape_images(keywords, num_images_per_keyword):
    for keyword in keywords:
        query = f'{keyword} images'
        search_results = search(query, num_results=10, lang='en')

        num_images_downloaded = 0
        for i, url in enumerate(search_results):
            if num_images_downloaded >= num_images_per_keyword:
                break

            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            image_elements = soup.find_all('img')
            for j, image in enumerate(image_elements):
                if num_images_downloaded >= num_images_per_keyword:
                    break

                image_url = image.get('src')
                if image_url:
                    if not image_url.startswith('http'):
                    continue  # Skip invalid URLs
                    
                    response = requests.get(image_url, stream=True)
                    file_path = f'{keyword}_{i}_{j}.jpg'
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(chunk_size=1024):
                            file.write(chunk)

                    print(f'Downloaded: {file_path}')
                    num_images_downloaded += 1

keywords = ['fruit', 'car', 'cakes']

num_images_per_keyword = 5

scrape_images(keywords, num_images_per_keyword)


Downloaded: fruit_0_0.jpg
Downloaded: fruit_0_1.jpg
Downloaded: fruit_0_2.jpg
Downloaded: fruit_0_3.jpg
Downloaded: fruit_0_4.jpg
Downloaded: car_0_0.jpg
Downloaded: car_0_1.jpg
Downloaded: car_0_2.jpg
Downloaded: car_0_3.jpg
Downloaded: car_0_4.jpg
Downloaded: cakes_0_0.jpg
Downloaded: cakes_0_1.jpg
Downloaded: cakes_0_2.jpg
Downloaded: cakes_0_3.jpg
Downloaded: cakes_0_4.jpg


In [47]:
#question #4

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_flipkart_smartphones(search_query):
    url = f"https://www.flipkart.com/search?q={search_query}&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_17_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_17_na_na_ps&as-pos=1&as-type=RECENT&suggestionId=oneplus+nord%7CMobiles&as-searchtext=oneplus%20nord"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    products = soup.find_all('div', {'class': '_1AtVbE'})

    results = []
    for product in products:
        details = {
            'Brand Name': '-',
            'Smartphone Name': '-',
            'Colour': '-',
            'RAM': '-',
            'Storage(ROM)': '-',
            'Primary Camera': '-',
            'Secondary Camera': '-',
            'Display Size': '-',
            'Battery Capacity': '-',
            'Price': '-',
            'Product URL': '-'
        }

        brand_name = product.find('div', {'class': '_4rR01T'})
        if brand_name:
            details['Brand Name'] = brand_name.text

        smartphone_name = product.find('a', {'class': '_1fQZEK'})
        if smartphone_name:
            details['Smartphone Name'] = smartphone_name.text

        colour = product.find('div', {'class': '_3BYOdg'})
        if colour:
            details['Colour'] = colour.text.split(":")[1].strip()

        price = product.find('div', {'class': '_30jeq3 _1_WHN1'})
        if price:
            details['Price'] = price.text

        product_url = product.find('a', {'class': '_1fQZEK'})
        if product_url:
            details['Product URL'] = "https://www.flipkart.com" + product_url['href']

        results.append(details)

    df = pd.DataFrame(results)
    df.to_csv('smartphones.csv', index=False)

    return df

search_query = 'Oneplus Nord'

df = scrape_flipkart_smartphones(search_query)

print(df)


                                        Brand Name  \
0                                                -   
1                                                -   
2                                                -   
3    OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)   
4   OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)   
5            OnePlus Nord 2T 5G (Jade Fog, 128 GB)   
6            OnePlus Nord 2T 5G (Jade Fog, 128 GB)   
7         OnePlus Nord 2T 5G (Gray Shadow, 128 GB)   
8                  OnePlus Nord (Gray Onyx, 64 GB)   
9               OnePlus Nord (Blue Marble, 256 GB)   
10              OnePlus Nord (Blue Marble, 128 GB)   
11                OnePlus Nord (Gray Onyx, 256 GB)   
12                OnePlus Nord (Gray Onyx, 128 GB)   
13        OnePlus Nord 2T 5G (Gray Shadow, 128 GB)   
14       OnePlus Nord CE 5G (Charcoal Ink, 128 GB)   
15         OnePlus Nord CE 5G (Silver Ray, 256 GB)   
16          OnePlus Nord CE 5G (Blue Void, 256 GB)   
17       OnePlus Nord CE 5G 

In [49]:
#question # 5

In [54]:
def extract_coordinates(url):
    url_parts = url.split('/')

    # Search for the part containing the coordinates
    for part in url_parts:
        if part.startswith('@'):
            # Extract the coordinates from the part
            coordinates = part[1:].split(',')
            latitude = coordinates[0]
            longitude = coordinates[1]
            return latitude, longitude

    return None

city = input("Enter the city name: ")

search_query = f"{city} coordinates"
search_results = search(search_query, num_results=1)

if search_results:
    url = search_results[0]
    coordinates = extract_coordinates(url)

    if coordinates:
        latitude, longitude = coordinates
        print(f"Coordinates for {city}: Latitude - {latitude}, Longitude - {longitude}")
    else:
        print("Coordinates not found for the given city.")
else:
    print("No search results found.")


Enter the city name: nellore


TypeError: 'generator' object is not subscriptable

In [ ]:
#question 6

In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_laptops():
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"

    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    laptops_containers = soup.find_all('div', {'class': 'product-listing'})

    brands = []
    names = []
    prices = []
    ratings = []

    for container in laptops_containers:
        brand = container.find('div', {'class': 'ProductSpecs'}).find('a').text.strip()
        name = container.find('div', {'class': 'ProductListingDetails'}).find('a').text.strip()
        price = container.find('div', {'class': 'ProductPrice'}).text.strip()
        rating = container.find('div', {'class': 'ValueRating'}).text.strip()

        brands.append(brand)
        names.append(name)
        prices.append(price)
        ratings.append(rating)

    data = {
        'Brand': brands,
        'Name': names,
        'Price': prices,
        'Rating': ratings
    }
    df = pd.DataFrame(data)

    return df

laptops_df = scrape_laptops()

print(laptops_df)

laptops_df.to_csv('gaming_laptops.csv', index=False)


Empty DataFrame
Columns: [Brand, Name, Price, Rating]
Index: []


In [61]:
#question 7

In [69]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_billionaires():
    url = 'https://www.forbes.com/billionaires/'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    container = soup.find('div', {'class': 'richlist'})
    if container is None:
        print("Unable to find billionaire data. Please check the URL or try again later.")
        return None

    ranks = []
    names = []
    net_worths = []
    ages = []
    citizenships = []
    sources = []
    industries = []

    rows = container.find_all('div', {'class': 'rank'})
    for row in rows:
        rank = row.find('div', {'class': 'rank-data'})
        ranks.append(rank.text.strip())

        name = row.find_next_sibling('div', {'class': 'personName'})
        names.append(name.text.strip())

        net_worth = name.find_next_sibling('div', {'class': 'netWorth'})
        net_worths.append(net_worth.text.strip())

        age = net_worth.find_next_sibling('div', {'class': 'age'})
        ages.append(age.text.strip())

        citizenship = age.find_next_sibling('div', {'class': 'countryOfCitizenship'})
        citizenships.append(citizenship.text.strip())

        source = citizenship.find_next_sibling('div', {'class': 'source-column'})
        sources.append(source.text.strip())

        industry = source.find_next_sibling('div', {'class': 'category'})
        industries.append(industry.text.strip())

    data = {
        'Rank': ranks,
        'Name': names,
        'Net worth': net_worths,
        'Age': ages,
        'Citizenship': citizenships,
        'Source': sources,
        'Industry': industries
    }
    billionaires_df = pd.DataFrame(data)

    return billionaires_df

billionaires_df = scrape_billionaires()

print(billionaires_df)


Unable to find billionaire data. Please check the URL or try again later.
None


In [70]:
#question 8

In [74]:
import os
import googleapiclient.discovery

API_KEY = 'YOUR_API_KEY'

VIDEO_ID = 'VIDEO_ID'

MAX_RESULTS = 500

youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=API_KEY)

response = youtube.commentThreads().list(
    part='snippet',
    videoId=VIDEO_ID,
    maxResults=MAX_RESULTS
).execute()

comments = []
for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
    upvotes = item['snippet']['topLevelComment']['snippet']['likeCount']
    time = item['snippet']['topLevelComment']['snippet']['publishedAt']
    comments.append({'comment': comment, 'upvotes': upvotes, 'time': time})

for comment in comments:
    print('Comment:', comment['comment'])
    print('Upvotes:', comment['upvotes'])
    print('Time:', comment['time'])
    print('---')


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=VIDEO_ID&maxResults=500&key=YOUR_API_KEY&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

In [73]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [75]:
#question 9

In [82]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.hostelworld.com/search?search_keywords=London,%20England&country=England&city=London&date_from=2023-06-20&date_to=2023-06-23&number_of_guests=1'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

print(soup)

<!DOCTYPE html>

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
<head>
<title>Hostels in London » Find cheap, unique hostels » Hostelworld</title><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1" data-n-head="ssr" name="viewport"/><meta content="yes" data-hid="mobile-web-app-capable" data-n-head="ssr" name="mobile-web-app-capable"/><meta content="hostelworld" data-hid="apple-mobile-web-app-title" data-n-head="ssr" name="apple-mobile-web-app-title"/><meta content="#ff7547" data-hid="theme-color" data-n-head="ssr" name="theme-color"/><meta content="website" data-hid="og:type" data-n-head="ssr" name="og:type" property="og:type"/><meta content="hostelworld" data-hid="og:title" data-n-head="ssr" name="og:title" property="og:title"/><meta content="hostelworld" data-hid="og:site_name" data-n-head="ssr" name="og:site_name" property="og:site_name"/><meta content="Compare cheap London hostels. Read 2019 revie

In [83]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.hostelworld.com/search?search_keywords=London,%20England&country=England&city=London&date_from=2023-06-20&date_to=2023-06-23&number_of_guests=1'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

hostel_containers = soup.find_all('div', class_='property-card')

hostel_names = []
distances = []
ratings = []
total_reviews = []
overall_reviews = []
privates_from_price = []
dorms_from_price = []
facilities = []
property_descriptions = []

for container in hostel_containers:
    hostel_name = container.find('h2', class_='title').text.strip()
    hostel_names.append(hostel_name)
    
    distance = container.find('span', class_='description').text.strip()
    distances.append(distance)
  
    rating = container.find('div', class_='score').text.strip()
    ratings.append(rating)
 
    total_review = container.find('div', class_='reviews').text.strip()
    total_reviews.append(total_review)

    overall_review = container.find('div', class_='keyword').text.strip()
    overall_reviews.append(overall_review)
    
    privates_from = container.find('span', class_='price-col').text.strip()
    privates_from_price.append(privates_from)
 
    dorms_from = container.find('span', class_='price-col-alt').text.strip()
    dorms_from_price.append(dorms_from)
    
    facility_tags = container.find_all('div', class_='facilities-label')
    facility_list = [tag.text.strip() for tag in facility_tags]
    facilities.append(', '.join(facility_list))
 
    description = container.find('div', class_='property-description').text.strip()
    property_descriptions.append(description)

data = {
    'Hostel Name': hostel_names,
    'Distance from City Centre': distances,
    'Rating': ratings,
    'Total Reviews': total_reviews,
    'Overall Review': overall_reviews,
    'Privates from Price': privates_from_price,
    'Dorms from Price': dorms_from_price,
    'Facilities': facilities,
    'Property Description': property_descriptions
}
df = pd.DataFrame(data)

df.to_csv('hostel_data.csv', index=False)

print(df)


AttributeError: 'NoneType' object has no attribute 'text'

In [84]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.hostelworld.com/search?search_keywords=London,%20England&country=England&city=London&date_from=2023-06-20&date_to=2023-06-23&number_of_guests=1'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

hostel_containers = soup.find_all('div', class_='property-card')

hostel_names = []
distances = []
ratings = []
total_reviews = []
overall_reviews = []
privates_from_price = []
dorms_from_price = []
facilities = []
property_descriptions = []

for container in hostel_containers:
    name_element = container.find('h2', class_='title')
    hostel_name = name_element.text.strip() if name_element else 'N/A'
    hostel_names.append(hostel_name)

    distance_element = container.find('span', class_='description')
    distance = distance_element.text.strip() if distance_element else 'N/A'
    distances.append(distance)
   
    rating_element = container.find('div', class_='score')
    rating = rating_element.text.strip() if rating_element else 'N/A'
    ratings.append(rating)
    
    total_review_element = container.find('div', class_='reviews')
    total_review = total_review_element.text.strip() if total_review_element else 'N/A'
    total_reviews.append(total_review)
  
    overall_review_element = container.find('div', class_='keyword')
    overall_review = overall_review_element.text.strip() if overall_review_element else 'N/A'
    overall_reviews.append(overall_review)

    privates_from_element = container.find('span', class_='price-col')
    privates_from = privates_from_element.text.strip() if privates_from_element else 'N/A'
    privates_from_price.append(privates_from)

    dorms_from_element = container.find('span', class_='price-col-alt')
    dorms_from = dorms_from_element.text.strip() if dorms_from_element else 'N/A'
    dorms_from_price.append(dorms_from)

    facility_tags = container.find_all('div', class_='facilities-label')
    facility_list = [tag.text.strip() for tag in facility_tags]
    facilities.append(', '.join(facility_list))
  
    description_element = container.find('div', class_='property-description')
    description = description_element.text.strip() if description_element else 'N/A'
    property_descriptions.append(description)

data = {
    'Hostel Name': hostel_names,
    'Distance from City Centre': distances,
    'Rating': ratings,
    'Total Reviews': total_reviews,
    'Overall Review': overall_reviews,
    'Privates from Price': privates_from_price,
    'Dorms from Price': dorms_from_price,
    'Facilities': facilities,
    'Property Description': property_descriptions
}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('hostel_data.csv', index=False)

print(df)


   Hostel Name Distance from City Centre Rating Total Reviews Overall Review  \
0          N/A                       N/A    N/A           N/A            N/A   
1          N/A                       N/A    N/A           N/A            N/A   
2          N/A                       N/A    N/A           N/A            N/A   
3          N/A                       N/A    N/A           N/A            N/A   
4          N/A                       N/A    N/A           N/A            N/A   
5          N/A                       N/A    N/A           N/A            N/A   
6          N/A                       N/A    N/A           N/A            N/A   
7          N/A                       N/A    N/A           N/A            N/A   
8          N/A                       N/A    N/A           N/A            N/A   
9          N/A                       N/A    N/A           N/A            N/A   
10         N/A                       N/A    N/A           N/A            N/A   
11         N/A                       N/A

In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.hostelworld.com/search?search_keywords=London,%20England&country=England&city=London&date_from=2023-06-20&date_to=2023-06-23&number_of_guests=1'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

hostel_containers = soup.find_all('div', class_='fabresult-inner')

hostel_names = []
distances = []
ratings = []
total_reviews = []
overall_reviews = []
privates_from_price = []
dorms_from_price = []
facilities = []
property_descriptions = []

for container in hostel_containers:
    name_element = container.find('h2', class_='fabresult-title')
    hostel_name = name_element.text.strip() if name_element else 'N/A'
    hostel_names.append(hostel_name)

    distance_element = container.find('span', class_='location')
    distance = distance_element.text.strip() if distance_element else 'N/A'
    distances.append(distance)

    rating_element = container.find('div', class_='score')
    rating = rating_element.text.strip() if rating_element else 'N/A'
    ratings.append(rating)

    total_review_element = container.find('div', class_='review')
    total_review = total_review_element.text.strip() if total_review_element else 'N/A'
    total_reviews.append(total_review)

    overall_review_element = container.find('span', class_='total-reviews')
    overall_review = overall_review_element.text.strip() if overall_review_element else 'N/A'
    overall_reviews.append(overall_review)

    privates_from_element = container.find('div', class_='price-col')
    privates_from = privates_from_element.text.strip() if privates_from_element else 'N/A'
    privates_from_price.append(privates_from)

    dorms_from_element = container.find('div', class_='price-col-alt')
    dorms_from = dorms_from_element.text.strip() if dorms_from_element else 'N/A'
    dorms_from_price.append(dorms_from)

    facility_tags = container.find_all('span', class_='facility-label')
    facility_list = [tag.text.strip() for tag in facility_tags]
    facilities.append(', '.join(facility_list))

    description_element = container.find('div', class_='desc')
    description = description_element.text.strip() if description_element else 'N/A'
    property_descriptions.append(description)

data = {
    'Hostel Name': hostel_names,
    'Distance from City Centre': distances,
    'Rating': ratings,
    'Total Reviews': total_reviews,
    'Overall Review': overall_reviews,
    'Privates From Price': privates_from_price,
    'Dorms From Price': dorms_from_price,
    'Facilities': facilities,
    'Property Description': property_descriptions
}

df = pd.DataFrame(data)

df.to_csv('hostel_data.csv', index=False)

print(df)


Empty DataFrame
Columns: [Hostel Name, Distance from City Centre, Rating, Total Reviews, Overall Review, Privates From Price, Dorms From Price, Facilities, Property Description]
Index: []
